<a href="https://colab.research.google.com/github/puruBHU/ColorHistogram/blob/master/HourglassHeatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp "/content/drive/MyDrive/Datasets/onehand10k.zip" './'
!unzip -q 'onehand10k.zip'

In [1]:
import numpy as np
import pandas as pd
from   pathlib import Path


import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint


# Local library import
from CustomDataloader import DataAugmentor
# from CustomCallback import PolyLR


from Model import LSTD

In [2]:
root         = Path.cwd()
train_file   = Path.cwd()/'onehand10k_train_files.txt'
val_file     = Path.cwd()/'onehand10k_test_files.txt'

In [3]:
target_size    = (256,) * 2

batch_size     = 16
epochs         = 150

n_cells        = target_size[0] // 32

In [4]:
train_data   = DataAugmentor(rescale         = 1/255.0,
                             horizontal_flip = False,
                             vertical_flip   = False,
                             translation     = False,
                             rotation        = False)

train_loader = train_data.flow_from_directory(root        = root,
                                              data_file   = train_file, 
                                              target_size = target_size,
                                              mode        = 'train',
                                              batch_size  = batch_size,
                                              shuffle     = True)


val_data    = DataAugmentor(rescale = 1/255.0)

val_loader  = val_data.flow_from_directory(root        = root,
                                           data_file   = val_file, 
                                           target_size = target_size,
                                           mode        = 'test',
                                           batch_size  = batch_size,
                                           shuffle     = False
                                           )


In [5]:
save_path = Path.cwd()/'checkpoints'

if not save_path.exists():
  save_path.mkdir(parents = True)
  
  
checkpoints = ModelCheckpoint('{}/LSTD_OneHand_SGD_'.format(save_path) + '{epoch:03d}_{val_loss:0.4f}.hdf5',
                              monitor        = 'val_loss',
                              mode           = 'auto',
                              verbose        = 1,
                              save_best_only = True, 
                              save_freq      = 'epoch'
                              )

# records     = CSVLogger('logs/LSTD_OneHand_training.log')
callbacks = [checkpoints]

In [6]:
def rmse(x, y):
    x = tf.math.sqrt(tf.keras.losses.MSE(x,y))
    return x

loss         = {'heatmapsA':rmse,
                'heatmapsB':rmse
                }

metrics      = {'heatmapsA': 'mae',
                'heatmapsB': 'mae'
               }

In [7]:
model    = LSTD(input_shape = target_size + (3,), n_keys = 21)
opt      = Adam(lr = 0.0002, beta_1 = 0.5)  
# opt      = SGD(lr = initial_lr, momentum=0.9, nesterov=True, decay=1e-6)

model.compile(optimizer    = opt,
              loss         = loss,
              metrics      = metrics,
              )

In [ ]:
model.fit(x                   = train_loader,
          validation_data     = val_loader,
          epochs              = epochs,
          steps_per_epoch     = len(train_loader),
          callbacks           = callbacks,
          use_multiprocessing = False,
          workers             = 4,
          verbose             = 1,
          initial_epoch       = 0
          )

Epoch 1/150
275/623 [============>.................] - ETA: 4:04 - loss: 0.6948 - heatmapsA_loss: 0.3253 - heatmapsB_loss: 0.3695 - heatmapsA_mae: 0.2581 - heatmapsB_mae: 0.2947